In [93]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [94]:
device

'cuda'

In [95]:
data_path = '../data/tiny_shakespeare.txt'
eval_iters = 200

In [96]:
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [97]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
n_embd = 32
print(''.join(chars))
print(VOCAB_SIZE)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [98]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda v: ''.join([itos[i] for i in v])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [99]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [100]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
BLOCK_SIZE = 8

In [101]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    #print(f'When input is {context} the target is: {target}')

In [105]:
torch.manual_seed(1337)
BATCH_SIZE = 32
BLOCK_SIZE = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')

print('----')

for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[b, :t+1]
        target = yb[b, t]
        #print(f'When input is {context.tolist()} the target is: {target}')

----


In [106]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [91]:
block_size = BLOCK_SIZE
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size, device=device)))
    
    def forward(self, x):
        b, t, c = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * c ** -0.5
        wei = wei.masked_fill(self.tril[:t, :t] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out

In [113]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [117]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.net(x)

In [118]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4).to(device)
        self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, VOCAB_SIZE)
    
    def forward(self, idx, targets=None):
        b, t = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device))
        x = tok_emb + pos_emb
        x = self.sa_head(x)
        x = self.ffwd(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape        
            logits = logits.view(b*t, c)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65]) tensor(4.2360, device='cuda:0', grad_fn=<NllLossBackward>)

..GDW.FyWjbm!UelJljnFAU;Qu.C-huDZqcdchvby:?Uup;Mnis'X3Uwty.HJlvBPUHIkyBf&pjY-lgvIEjVk:DmyqwJdlNBtSkk


In [119]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
epochs = 10000

In [120]:
for epoch in range(epochs):
    if epoch % 1000 == 0:
        losses = estimate_loss()
        print(f'step {epoch}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

step 0: train loss 4.2318, val loss 4.2328
step 1000: train loss 2.4487, val loss 2.4420
step 2000: train loss 2.3341, val loss 2.3442
step 3000: train loss 2.2857, val loss 2.2875
step 4000: train loss 2.2524, val loss 2.2634
step 5000: train loss 2.2054, val loss 2.2345
step 6000: train loss 2.1901, val loss 2.2328
step 7000: train loss 2.1686, val loss 2.2131
step 8000: train loss 2.1633, val loss 2.2108
step 9000: train loss 2.1509, val loss 2.2008
1.9505515098571777


Ko Winso wher eiings to th dour will nowired thoughower; the the danteruft for ar irier my to thy a


## Attention Mechanism

In [35]:
torch.manual_seed(1337)
b, t, c = 4, 8, 32
x = torch.randn(b, t, c)

# single head
head_size = 16
key = nn.Linear(c, head_size, bias=False)
query = nn.Linear(c, head_size, bias=False)
value = nn.Linear(c, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

tril = torch.tril(torch.ones(t, t))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

out.shape

In [37]:
wei.var()

tensor(0.1201, grad_fn=<VarBackward>)

In [ ]:
xbow = torch.zeros((b,t,c))
for i in range(b):
    for j in range(t):
            xprev = x[i, :j+1]
            xbow[i,j] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(t, t))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

In [ ]:
tril = torch.tril(torch.ones(t, t))
wei = torch.zeros((t, t))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)